# Fire Truck Project Report

Group members: Antoine Hinary, Francesco Maglie, Antoine Tissot-Favre, Armance Nouvel <br>
GitHub repo: https://github.com/fra-mgl/BOMR-project

## Table of Contents

1. [Introduction](#introduction)
2. [Vision Module](#vision-module)
   - 2.1 [Overview](#overview)
   - 2.2 [Environment description](#environment)
   - 2.3 [Module structure](#vision-structure)
   - 2.4 [Algorithms explanation](#vision-algorithms)
      - 2.4.1 [`vision_init()` function](#vision-init)
      - 2.4.2 [`get_thymio()` function](#get-thymio)
    - 2.5 [References](#vision-references)

3. [Optimal Path Computation](#optimal-path-computation)
   - 3.1 [Pathfinding Precision](#pathfinding-precision)
   - 3.2 [A* Algorithmic Mastery](#a-algorithmic-mastery)
   - 3.3 [Navigational Options](#navigational-options)
   - 3.4 [Path Reconstruction Elegance](#path-reconstruction-elegance)

4. [Motion Control Implementation](#motion-control-implementation)
   - 4.1 [Initialization and Configuration](#initialization-and-configuration)
   - 4.2 [Sensor Data Retrieval and Actuation](#sensor-data-retrieval-and-actuation)
   - 4.3 [Trajectory Following Mechanism](#trajectory-following-mechanism)

5. [Local Navigation Strategy](#local-navigation-strategy)
   - 5.1 [Assumptions](#assumptions)
   - 5.2 [Operational Assumptions](#operational-assumptions)
   - 5.3 [Obstacle Detection and Handling](#obstacle-detection-and-handling)
   - 5.4 [Maneuvering Strategies](#maneuvering-strategies)

6. [Kalman Filter in Localization](#kalman-filter-in-localization)
   - 6.1 [Adaptive Position Estimation](#adaptive-position-estimation)
   - 6.2 [Compensating for Uncertainties](#compensating-for-uncertainties)


## **1. Introduction<a name="introduction"></a>**

This project aims to integrate vision, path planning, local navigation, and filtering to guide a Thymio robot on a map towards a goal. We have developed two implementations: one guided by vision and another without relying on vision. The main goal is to create a robotic system with advanced features that is comparable to an autonomous fire rescue truck. 

In the initial setup, a webcam captures the surroundings of the experimental area. Real-time processing using traditional image techniques extracts essential map details like the robot's position, the map itself, fixed obstacles, and the destination. The A* algorithm then calculates the best route, relaying instructions to the global controller of the Thymio robot. This controller, in turn, directs the motors to follow the optimal path. If the Thymio detects an obstacle ahead through its proximity sensors, local navigation takes charge, guiding the robot to avoid collisions. Additionally, even if the Thymio is moved unexpectedly, it can still autonomously navigate its way to the intended destination.

Our project's story is based on a simulated scenario in which an autonomous fire rescue robot sets out on a mission to locate and rescue people trapped in a burning building, taking inspiration from the urgent and essential nature of firefighting scenarios. This hypothetical scenario highlights the necessity for robotic systems to manoeuvre through dangerous situations, assess the situation, and carry out precision rescue operations. It also matches the urgency and complexity of actual firefighting operations where speed and efficiency are keys for a successful salvation of the endangered victims.

### **Structure**

In our project's organisational structure, a central "Command Centre" assumes a pivotal role as the hub for coordinating the cooperation of many modules, ensuring the seamless functioning of our self-contained fire rescue robot. This crucial module takes on the tasks of establishing the Thymio connection, carrying out careful camera calibration to maximise colour and edge detection, calculating the grid map, and drawing the route for navigation. This command centre is modular in operation and handles critical robot duties such as taking quick decisions regarding the presence and absence of an obstacle. The robot operates in two different modes: local navigation, which allows for real-time obstacle avoidance, and global navigation, which plots a route to the closest checkpoint. While maintaining knowledge of the position and orientation of the robot using either the Camera module or the Kalman filter. The diagram presented below illustrates the overarching structure of our project.

</div style="width: 30%;">
    <img src="./img/general_diagram.jpeg" />
    <style="width: 100%;">
    <p style="text-align: center;">Figure 2: General Diagram of the overall structure of the project</p>
</div>



### **Modules**

#### **Robot State Estimation with Extended Kalman Filter**

In our autonomous ambulance rescue robot project, accurate position and speed estimation are vital for effective navigation. We utilize an Extended Kalman Filter (EKF) implemented in the `KalmanFilter` class. The state space representation is tailored to our specific needs, considering a 4-dimensional state vector $([ \text{x}, \text{y}, \text{v}_x, \text{v}_y ])$, where $(\text{v}_x)$ and $(\text{v}_y)$ represent the linear velocities in the x and y directions, respectively.


##### **Coordinate System**

The coordinate system follows the convention shown below.

Here, $(x)$ and $(y)$ are in centimeters, and $(\text{v}_x)$ and $(\text{v}_y)$ are in centimeters per second.

##### **Discrete Model**

The discrete motion model for the Kalman filter is given by:

\[
\begin{align*}
\mathbf{z}_{k+1} &= f(\mathbf{z}_k) \\
\mathbf{z}^+ &= \begin{bmatrix} x + v_x * T_s \\ y + v_y * T_s \\ v_x \\ v_y \end{bmatrix} 
\end{align*}
\]

where $(\text{v}_x)$ and $(\text{v}_y)$ are computed as
$$
\begin{align*}
\text{v}_x = \dfrac{v_r + v_l}{2} * \cos \theta * \alpha \\
\text{v}_y = \dfrac{v_r + v_l}{2} * \sin \theta * \alpha
\end{align*}
$$

where $v_r$ and $v_l$ are the motor speeds (in RPM) and $\alpha$ is the speed conversion factor.<br>
Then, to compute the angle the robot has in the followin instant in time (ie. $\hat{\theta}$), we performe
$$
\begin{align*}
\omega = \dfrac{v_r - v_l}{d} * \alpha \\
\Delta \theta = \omega * T_s \\
\hat{\theta} = \theta + \Delta \theta
\end{align*}
$$


##### **Measurement Model**

The discrete measurement model is given by:

\[
\begin{align*}
\mathbf{z}_{k+1} &= h(\mathbf{z}_k) \\
\mathbf{z}^+ &= \begin{bmatrix} x_{\text{{measured}}} \\ y_{\text{{measured}}} \\ v_{x \text{{ measured}}} \\ v_{y \text{{ measured}}} \end{bmatrix} 
\end{align*}
\]

The measurement model incorporates data from the camera and the motor encoders, providing measured values for position and motor speeds. The covariance matrix $\mathbf{R}$ captures the measurement noise.

##### **Extended Kalman Filter Algorithm**

The steps of the extended Kalman filter algorithm, tailored to our implementation, are as follows:

1. **Prediction Step:**
In this step, the Kalman filter predicts the next state of the system based on the previous state estimate and the system dynamics, incorporating the process model. The predicted state and covariance represent the system's expected behavior, considering motion and process uncertainties :

   \begin{align*}
   \hat{\mathbf{x}}_{k \mid k-1} &= f(\hat{\mathbf{x}}_{k-1 \mid k-1}, \mathbf{u}_k) \\
   \mathbf{P}_{k \mid k-1} &= \mathbf{F}_k \mathbf{P}_{k-1 \mid k-1} \mathbf{F}_k^T + \mathbf{Q}_k
   \end{align*}

2. **Measurement Residual:**
The measurement residual calculates the difference between the predicted measurement (obtained from the predicted state) and the actual measurement. It represents the discrepancy or error between the expected and observed values, capturing the system's deviation from the predicted state : 

   \begin{align*}
   \tilde{\mathbf{y}}_k &= \mathbf{z}_k - h(\hat{\mathbf{x}}_{k \mid k-1})
   \end{align*}

3. **Measurement Update:**
In the measurement update step, the Kalman filter refines its state estimate by incorporating the measured values. The Kalman gain is calculated to give more weight to reliable measurements and less weight to uncertain ones. The updated state estimate and covariance reflect a more accurate representation of the system's true state, considering both prediction and measurement inform : 

   \begin{align*}
   \mathbf{S}_k &= \mathbf{H}_k \mathbf{P}_{k \mid k-1} \mathbf{H}_k^T + \mathbf{R}_k \\
   \mathbf{K}_k &= \mathbf{P}_{k \mid k-1} \mathbf{H}_k^T (\mathbf{S}_k)^{-1} \\
   \hat{\mathbf{x}}_{k \mid k} &= \hat{\mathbf{x}}_{k \mid k-1} + \mathbf{K}_k \tilde{\mathbf{y}}_k \\
   \mathbf{P}_{k \mid k} &= (\mathbf{I} - \mathbf{K}_k \mathbf{H}_k) \mathbf{P}_{k \mid k-1}
   \end{align*}


##### **Estimation of Ts (Time difference)**

The accurate estimation of the time between iterations (\(T_s\)) is essential for the dynamic performance of the Kalman filter, directly influencing temporal alignment, motion prediction, and covariance matrix updates. A precise \(T_s\) ensures that predictions align with the system's temporal dynamics, maintaining the filter's adaptability to dynamic changes. Inaccuracies in \(T_s\) can disrupt temporal alignment, affecting the filter's ability to provide accurate state estimates and impacting control actions.

##### **Estimation of Noise Matrices**

For the motion model:<br />
$$
Q=\begin{bmatrix}
0.04 & 0 & 0 & 0\\
0 & 0.04 & 0 & 0\\
0 & 0 & 6 & 0\\
0 & 0 & 0 & 6
\end{bmatrix}
$$

We derived the values of 0.04 and the angle through empirical estimation, without a specific deterministic basis. Regarding the speed, empirical data obtained during the eighth week informed our estimation process, wherein half of the variance was attributed to the inherent motion dynamics, while the remaining half originated from measurement uncertainties. <br>

For the measurement model:<br />
$$
R=\begin{bmatrix}
0.25 & 0 & 0 & 0\\
0 & 0.25 & 0 & 0\\
0 & 0 &6 & 0\\
0 & 0 & 0 & 6
\end{bmatrix}
$$

These values are estimated, with half of the variance attributed to motion and the other half to measurement.


## 2. Vision Module<a name="vision-module"></a>

### 2.1 Overview<a name="overview"></a>

The vision module serves as the visual intelligence of the Thymio robot, forming a critical component within the broader project framework. Its primary purpose revolves around interpreting the surrounding environment recognizing specific visual elements. The objectives of this module include determining the robot's position and all the order elements in the environment.

The OpenCV library is used to performe these visual algorithms.<br>
To be able to use the ArUco library, you should install a specific version of OpenCV. To do so, you can run these commands:<br>

        pip uninstall opencv-python
        pip install opencv-contrib-python==4.6.0.66


### 2.2 Environment description<a name="environment"></a>

The environment is defined by a white grid of arbitrary dimensions and each cell has the same dimention as the Thymio robot. To identify the grid, we use 4 ArUco markers placed on the corners of it.
We then define 3 types of objects that can be detected in the environment; these are squares and each should be not larger than a cell of the grid. The objects categories are<br>
- the static obstacles, which are in red
- the middle target, which is in blue
- the final target, which is in green

Lastly, to estimate the pose of the robot, we use another ArUco marker.<br>

We made the choice of using the ArUco markers because they are a very reliable way to extract the position of an object; moreover it's easy to find documentation on how to use them since they are widely use in a lot of computer vision applications.<br>
Then, the simple shape of the objects and the pronounced difference in the colors let the user easily identify them without the need of using other markers. 

</div style="width: 30%;">
    <img src="./img/env_map.jpg" />
    <style="width: 100%;">
    <p style="text-align: center;">Map setup </p>
</div>

### 2.3 Module structure<a name="vision-structure"></a>

All the code responsible for the visual feature extraction is placed in the module `vision`. The main components of this module are two classes, used to describe the visual elements, and two functions, which are called in the main loop of the code to performe the visual tasks.

The two classes that have been defined are (in file VisionObjects.py):<br>
- `VisionObject`: to handle a generic element in the environment. Then we define 3 subclasses, one for each caterogy of objects in the environment
- `Thymio`: a specific class to handle all the tasks directly associated with the Thymio

Then, in the main loop, we need to call two functions from the Vision module:
- `vision_init()`: to extract the grid and all the visual elements 
- `get_thymio()` : to extract the Thymio's pose

### 2.4 Algorithm explanation<a name="vision-algorithms"></a>

#### 2.4.1 `vision_init()` function<a name="vision-init"></a>

Let's begin by delving into the `vision_init()` function to understand what are the main steps that are performed (in parenthesis, one can find a reference to the file that contains such function).

1. Grid extraction (vision.detection.grid_extraction)<br>
    As said before, we used 4 ArUco markers to identify the corners and extract the grid. To identify such markers, we use the built-in functions in the OpenCV library.<br>
    Once defined the type of ArUco we are using, we just need to use the following snipped of code to detect them:<br>
    
        arucoDict = cv.aruco.Dictionary_get(cv.aruco.DICT_5X5_1000)
        arucoParams = cv.aruco.DetectorParameters_create()
        positions, ids, rejected = cv.aruco.detectMarkers(image, arucoDict, parameters=arucoParams)
    

    Since each marker has a specific ID, we can exploit this information to know which marker is in which corner and then extracting the grid.

2. Perspective transformation (vision.detection.perspective_transform)<br>
	We then need to correct the perspective of the image. To do so, we use the built-in function that compute a linear transformation from the original image to the corrected one (`matrix M` which is 3x3). Then we apply it to each pixel as following

    $$(\hat{x}, \hat{y}, \alpha)^T = M * (x, y, 1)^T$$
    where $(x, y)$ are the coordinates of the original pixel. To get the coordinates of the transformed one, $\hat{x}$ and $\hat{y}$ are divided by $\alpha$, yielding
    
    $$ (\dfrac{\hat{x}}{\alpha}, \dfrac{\hat{y}}{\alpha}) $$

    The snipped of code to achieve this is the following:

        M = cv.getPerspectiveTransform(src_points, dst_points)
        dst = cv.warpPerspective(source, M, dst_dim)

    where `dst` is the corrected image.
        

3. Recognition (vision.functions.recognition)<br>
To recognise the objects (given the fact they have different colours), we apply this algorithm for each class of objects (ie. for each color):<br>

- extract a mask for the given color (vision.detection.colored_object_extraction)<br>
                a. change colorspace from RGB to HSV (in this space it’s easier to detect colours)<br>
                b. compute the mask for the color we are looking for<br>
                c. performe a morphology opening (erosion followed by dilation): the objective of this step is to remove small elements (usually generated by noise)<br>
                d. find contours with OpenCV built-in function; it returns a list of points belonging to the contour of the object <br>

                contours, _ = cv.findContours(thresh, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)

- given the contours, we create one instance for each object and we compute the coordinates of each one in the grid.<br>
    The class `VisionObject` defines two methods to achieve this (vision.VisionObjects)<br>
        - `find_center()`: the mean on x and y of each point found by the `findContours()` function<br>
        - `compute_grid_coordinates()`: knowing the dimension of the grid (in pixel) and the number of cells, we compute the position in the grid<br>

                y = floor(self.center_pixel[0] / grid_width_pixel * grid_width_cells)
                x = floor(self.center_pixel[1] / grid_height_pixel * grid_height_cells)
                    
- compute the output of the function<br>
        Knowing the position of each obstacle, we can create an occupancy grid, ie. a grid in which we set 1 if there is an obstacle (occupied cell), 0 otherwise.<br>
        For the middle and final targets, we define them as lists.


#### 2.4.2 `get_thymio()` function<a name="get-thymio"></a>
Let's now take a look to the `get_thymio()` function, which is used to retrive the pose of the robot, ie. x and y coordinates and the orientation angle.

1. Grid extraction and perspective correction (as before)

2. `thymio_recognition()` (vision.function.thymio_recognition)
	- find the ArUco marker as before, but this time we just need to detect the Thymio's marker, which is defined by `id = 5`
	- performe `thymio.pose_estimation()`<br>

                thymio.pose_estimation(env.shape[1], env.shape[0])
        Now we need to find the coordinates and the orientation of the robot. To compute the coordinates, we performe the same as before to compute the coordinates of each object in the environment<br>

		        self.find_center()
                self.compute_grid_coordinates(grid_width_pixel, grid_height_pixel)
	    To compute the orientation angle, we use the function `compute_theta()`
    
                theta = self.compute_theta()
        Using the ArUco library, we can detect the four corners of the marker. Knowing them, we compute the two line connecting the corners’ pairs 0-3 and 1-2 and the corresponding angle. To do that, we exploit the relation between the angle and the angular coefficient of the line:

        $$ \theta = arctan2(\Delta y, \Delta x)$$

        The use of the function $arctan2$ ensures us to have $\theta \in (- \pi, + \pi]$
                
        We compute both angles and then we take the average in order to be more robust to noise and be more precise.<br>
        The last thing we need to do is a change of reference: since we are using pixels to compute the lines and thus the angle, the image frame and the one used by the Thymio to describe the environment are not the same. The transformation is the following:
        $$ \hat{\theta} = - \theta + 90 $$
        and it is performed inside the `fix_theta()` function.<br>
        A figure is provided below to visualize the difference between the two references.

</div style="width: 30%;">
    <img src="./img/angle_reference.jpg" />
    <style="width: 100%;">
    <p style="text-align: center;">Reference frames </p>
</div>

    
### 2.5 References<a name="vision-references"></a>
OpenCV: https://opencv.org<br>
ArUco markers:<br>
    https://pyimagesearch.com/2020/12/21/detecting-ArUco-markers-with-opencv-and-python/<br>
    https://github.com/niconielsen32/ComputerVision/tree/master/ArUco<br>
Perspective transform: https://medium.com/analytics-vidhya/opencv-perspective-transformation-9edffefb2143<br>
Morphology opening: https://docs.opencv.org/3.4/d9/d61/tutorial_py_morphological_ops.html<br>
Color extraction: https://docs.opencv.org/3.4/df/d9d/tutorial_py_colorspaces.html<br>
Find contours: https://docs.opencv.org/3.4/d4/d73/tutorial_py_contours_begin.html




## 3. Optimal Path Computation<a name="optimal-path-computation"></a>
The A algorithm was implemented using the exercise set 5 of the course, it was adapted to fit our environment (a rectangular grid) by adding `max_val_x` and `max_val_y` values. 
### 3.1 Pathfinding Precision<a name="pathfinding-precision"></a>

This section delves into global navigation, unveiling the Thymio's optimal path computation facilitated by the A* algorithm. Beyond simple movement, this algorithmic approach strives to map the most efficient course for the robot through a predefined grid, minimizing the cost to achieve the desired result.

### 3.2 A* Algorithmic Mastery<a name="a-algorithmic-mastery"></a>

The A* algorithm, pivotal in our pathfinding strategy, involves key steps:
Initialization establishes the start and goal.
A heuristic guides efficient exploration.
Management of open and closed sets optimizes node evaluation.
Cost functions track path costs, aiding optimal route decisions.
Defined movement options dictate allowable transitions and costs.
The main loop iteratively evaluates nodes until reaching the goal or depleting the open set.
Node expansion assesses neighbors, updating paths if more cost-effective options are found.
Path reconstruction constructs the optimal path upon goal attainment.

### 3.3 Navigational Options<a name="navigational-options"></a>

Two sets of movement options, 4-connectivity and 8-connectivity, are available within the A* algorithm. Opting for 4N connectivity aligns with practical constraints, ensuring computational efficiency in navigating the constrained layout of the simulated environment, mirroring realistic scenarios with restricted diagonal movements.

### 3.4 Path Reconstruction Elegance<a name="path-reconstruction-elegance"></a>

Efficient path reconstruction in the A* algorithm's code ensures the Thymio navigates carefully and reaches its destination. Utilizing the reconstruct_path function, the algorithm traces back each step, creating a list of visited nodes for the optimal path. This list, containing nodes in order from start to goal, serves as a smooth roadmap for the Thymio's precise navigation.

## 4. Motion Control Implementation Overview<a name="motion-control-implementation-overview"></a>

### 4.1 Initialization and Configuration<a name="initialization-and-configuration"></a>

The motion control process initiates with the instantiation of the `Control` class, where parameters for motion control, including the proportional (kp), integral (ki), and derivative (kd) gains, as well as the robot's wheel base width, are defined. These parameters establish the foundation for subsequent motion control strategies. Additionally, the class contains methods for sensor data retrieval and actuation, contributing to the overall adaptability of the robot's motion.

### 4.2 Sensor Data Retrieval and Actuation<a name="sensor-data-retrieval-and-actuation"></a>

The `Control` class features a method named `get_sensors` responsible for retrieving sensor data such as motor speeds and horizontal proximity values. This method provides essential information for making informed decisions during the motion control process and also for the local avoidance tasks. The `set_motors_PID` method is implemented to adjust the left and right motor speeds based on PID control, enhancing the adaptability and responsiveness of the robot to environmental changes and thus limiting the angular error allowed for the robot.

### 4.3 PID Controller Implementation<a name="pid-controller-implementation"></a>

A PID controller is implemented within the `pid_controller` function, utilizing proportional, integral, and derivative terms to dynamically adjust motor speeds. The control gains (kp, ki, kd) and the robot's width play a crucial role in determining the correction applied to the motors. The function also includes a mechanism to handle aggressive corrections for significant angle deviations, ensuring effective turning when needed. The `normalize_angle` function is utilized to maintain the angle within the range of -180 to 180 degrees.

**Table of Constants for Motion Control:**

| Constant        | Value/Description                          |
|-----------------|--------------------------------------------|
| kp              | 2.0                                        |
| ki              | 0.04                                       |
| kd              | 0.01                                       |
| robot_width     | 0.1                                        |

This set of constants is meticulously chosen to balance the responsiveness and stability of the robot's motion control, ensuring effective navigation in various scenarios. The proportional, integral, and derivative gains are fine-tuned to achieve optimal performance, while the robot's width is considered for accurate motion calculations.


## 5. Local Navigation Strategy<a name="local-navigation-strategy"></a>

Reference : For this module, we utilize the lab of the course about the local navigation.

The Local Navigation module is designed to respond dynamically to obstacles detected by the Thymio robot's proximity sensors. The goal is to trigger Local Navigation when an obstacle is detected, ensuring the robot can navigate around it effectively. Proximity sensors' captured values trigger Local Navigation when above a desired threshold distance.
 
 - Key parameters :

| Variables              | Meaning                                                      | Type                 |
|------------------------|--------------------------------------------------------------|----------------------|
| `sens`                 | The initial direction of movement ('Right' or 'Left')        | string               |
| `obstacle`             | Boolean indicating whether an obstacle is detected           | bool                 |
| `special_step`         | The determining special case from the list of `special_cases`| int                  |
| `status`               | Index of the step in the path after the obstacle handling    | int                  |
| `orientation`          | Orientation of the Thymio                                    | int                  |
| `special_cases`        | List of 2 cases that need specific handling during obstacles | int array (1*2)      |
| `k`                    | Index to determine which point of the path we are taking     | int array ()         |
| `path`                 | Global path of the Thymio                                    | (type not specified) |
| `position`             | Position of the Thymio in front of the obstacle              | int array (1*2)      |
| `prox[]`               | List of all values of the proximity sensors                  | float array          |

 - Key functions : 

| Function                   | Input                                          | Output                                       |
|----------------------------|------------------------------------------------|----------------------------------------------|
| `local_nav`                | (An instance of the Thymio robot class)      | boolean indicating whether an obstacle is detected              |
| `obstacle_extraction`      | (grid of obstacles)                            | list of obstacle's coordinates               |
| `obstacle_function`        | (path, position, angle, thymio, obstacle_coordinates)| index of the next step in path              |
| `handle_final_orientation` | (path, k, axe, thymio)                         | updated state after handling the final orientation                     |
| `rotate_robot`             | (angle_degrees, rotation_speed, sens, thymio) | (no output)                     |
| `forward_robot`            | (motor_speed, distance, thymio)                | (no output)                      |
| `determine_special_path`   | (path, special_cases, axe, special_step)      | The updated value of `special_step`                    |


The diagram showcases the architecture of our "Local Navigation" module, a critical component enabling real-time obstacle avoidance and autonomous navigation. Through interconnected sub-modules, it orchestrates obstacle detection, path planning, and motor control, ensuring the robot navigates dynamically and adapts to its environment.

</div style="width: 30%;">
    <img src="./model_local_nav.jpeg" />
    <style="width: 100%;">
    <p style="text-align: center;">Figure 3: Local Navigation System Operation Diagram</p>
</div>

### 5.1 Assumptions<a name="local-assumptions"></a>

In our environment, some obstacles are strategically placed along the robot's path. The Local Navigation module operates under the following assumptions:

1. **Obstacle Blocking Assumption:**
   - The obstacles are completely blocking the robot's path. This choice aligns with our decision to perform Global Navigation only once at the program's beginning. This assumption allows Local Navigation to focus on avoiding specific obstacles without being constrained by the assumptions made during Global Navigation and thus releaving us from the necessity of recomputing the global path

2. **Light Reflective Obstacle Assumption:**
   - The obstacles are assumed to be light-reflective as the proximity sensors utilize infrared light. The sensors consist of a transmitter and a receiver, with the light reflecting off the obstacle. This assumption rules out obstacles that diffract light extensively, such as glass, ensuring reliable detection.

3. **Simple Convex Geometrical Section Assumption:**
   - Obstacles are assumed to have a simple convex geometric shape including triangles, rectangles and squares. This assumption facilitates the robot's ability to navigate around obstacles by following specific scenarios preimplemented. It ensures that the robot can circumvent obstacles without encountering complex concave shapes that might hinder navigation.

### 5.2 Operational Assumptions<a name="local-operational-assumptions"></a>

Given these assumptions, the Local Navigation module specifically assumes that obstacles are square-shaped. When an obstacle is detected, the entire square is considered impassable, and the robot plans its path accordingly. The assumption further specifies that the obstacle is positioned centrally within the square.

This operational approach enables the robot to navigate around square obstacles effectively. However, it is important to note that the Local Navigation module does not require adjustements if different obstacle shapes are introduced due to the designed library of local paths to follow. The successful execution of Local Navigation relies on the accurate adherence to these assumptions for obstacle characteristics and the geometric layout of the environment.

### 5.3 Obstacle Detection and Handling<a name="obstacle-detection-and-handling"></a>


**Obstacle Identification:** 
The robot retrieves and scales data from the proximity sensor to identify new obstacles in its path that may not have been captured by the camera. The `local_nav()` function checks if the proximity sensor in front of the robot, denoted as `x[2]`, does not detect a new obstacle. Upon the threshold of 20 being exceeded by the value of `x[2]`, the system switches to the obstacle-handling state, triggering the `obstacle_function()` when the boolean variable `obstacle`is set to true.

**Obstacle Handling:**
 When an obstacle is encountered, the robot stops and assesses the position relative to the global path. Three scenarios are considered:
  - If the global path is behind the obstacle, the robot maneuvers to reach the path (`state = obstacle_behind`).
  - If the target is to the left of the obstacle, the robot executes a specific maneuver (`state = obstacle_left`).
  - If the target is to the right of the obstacle, the robot follows a different maneuver (`state = obstacle_right`). 
To determine the scenario, we conduct a comparison between the robot's position in front of the obstacle (`position[axe]`) and the subsequent position in the global path (`path[axe][1]`), taking into consideration the abscissae of the positions. The comparison coordinate is determined by the Thymio's orientation in front of the obstacle:
  - If the orientation is 180 or 0 degrees, `axe` is set to 1.
  - If the orientation is 90 or -90 degrees, `axe` is set to 0.


### 5.4 Maneuvering Strategies<a name="maneuvering-strategies"></a>

Depending on the selected senario, we will apply different maneuvering strategies. When an obstacle is encountered, the robot dynamically decides whether to turn left or right based on its initial target position relative to the obstacle. This decision influences the choice of one of the following functions. This approach enables the robot to adapt its maneuvering strategy, ensuring efficient obstacle avoidance and alignment with the next best location of the global path :

- **Maneuvering Behind Obstacle (`obstacle_behind`):** 
    `handle_target_behind(special_step, "Right", k, thymio,obstacle_coordinates,position,orientation, axe)`
- **Maneuvering Left of Obstacle (`obstacle_left`):**
    `handle_target_left(thymio)`
- **Maneuvering Right of Obstacle (`obstacle_right`):**
    `handle_target_right(thymio)`
When the target is situated behind an obstacle, two scenarios merit consideration:

- If the target is behind the obstacle (case 2 on the sketch above), and the subsequent step in the global path does not align with the predefined special cases, the robot straightforwardly moves to the position behind the obstacle. The thymio dynamically adjusts its trajectory based on current obstacles. The `obstacle_extraction(obs_grid)` function extracts a list of obstacle coordinates, which is then compared with the robot's position to determine obstacles on its left or right. If detected, the `handle_target_behind` function adapts the path accordingly.

- If the target is behind the obstacle, and the next step in the global path corresponds to either of the two special cases previously defined (cases 1 and 3 on the sketch above), it is more optimal for the robot to navigate directly to the second position in the global path, precisely on the special cases.

Finally, once the obstacle is navigated, we enter in `state = orientation`. The robot assesses the next step in the global path to adjust its orientation with the next step in the path. To decide if it has to turn or not we just compare the abscisse of the robot and the one of the next position in the path.

- **Next Step (`next_step`):** 
    `handle_final_orientation(path, k,axe, thymio)`
   

## 6. Kalman Filter in Localization<a name="kalman-filter-in-localization"></a>

### 6.1 Adaptive Position Estimation<a name="adaptive-position-estimation"></a>

The Kalman Filter is a pivotal component in our localization strategy, adapting to changing conditions and continuously refining its estimates based on motor speed inputs. We use the kalman mainly in the case of camera obscurity or perturbation to send the estimated position and angle of the robot to the control functions in real time. The Kalman is structures as follows : first the functoin called **compute_x_y_speed(self, left_motor_speed, right_motor_speed)** is called taking as inputs the left and right motor speeds, it then computes the speeds in term of linear speed (v_r - v_l)/2 , the angular speed, the the vectorial speeds [v_x and v_y] and finally uses the difference between the 2 wheels' speed to compute the angular variation $\Delta\theta$ to estimate the actual orienation and angle of the our Robot.

In our state space representation, we chose not to include the angle (\(\theta\)) as a primary state variable to enhance computational efficiency. Instead, we compute and estimate \(\theta\) separately using the motor inputs from sensor values. This modular approach streamlines computations, reduces complexity, and aligns with the specific requirements of our application. In addition, the speeds returned from the functions are then being passed through our extended Kalman Filter to ultimately conclude our estimation for the position which is the most important element for which we have the estimator. In addition to our global choices, we have decided to facilitate the readability of the filtering task and merge both steps explained above for this task : the **prediction** and the **update** in a single predict function that outputs the expected status of the system and the updated matrix **P** as well.

#### Constants used for the filtering 

| Constant                   | Value or Description            | Justification                                                                |
|-----------------------------|--------------------------------|------------------------------------------------------------------------------|
| `dt`                        | 0.08                                          | Time step for the state transition matrix by default untill the comuted time is passed to the function                    |
| `speed_conv_factor`         | 0.35                                           |Conversion factor for motor speed to linear speed computed by observing the robot for a distance of 12cm with a speed ot 80 and computing the \(\delta t\) thus extracting the factor             |
| `wheel_width`               | 100 (distance between the two wheels in mm)    | Distance between the two wheels of the robot used to compute the angular speed of the robot             |


```python
class KalmanFilter(object):
    def __init__(self, dim_x=4, dim_z=2, dt=0.1):
        # Initialization code

    def compute_x_y_speed(self, left_motor_speed, right_motor_speed):
        # Compute linear speed and angular speed
        return speed, new_angle
    
    def predict(self, left_motor_speed, right_motor_speed, orientation, x_est_prev, P_est_prev):
        # Measurement prediction covariance
        S = np.dot(self.H, np.dot(self.P_est, self.H.T)) + self.R

        # Kalman gain
        K = np.dot(self.P_est, np.dot(self.H.T, np.linalg.inv(S)))

        # A posteriori estimate
        self.x_est = self.x_est + np.dot(K, i)
        self.P_est = self.P_est - np.dot(K, np.dot(self.H, self.P_est))

        # Extracting relevant information for the output...
        return estimated_position, estimated_speed, angle, self.x_est, self.P_est
```

1. **`__init__(self, dim_x=4, dim_z=2, dt=0.1):`**
   - **Purpose:** Initializes the Kalman filter object with specified dimensions and time parameters.

2. **`compute_x_y_speed(self, left_motor_speed, right_motor_speed):`**
   - **Purpose:** Computes the linear speed (in x and y directions) and angular speed of the robot based on left and right motor speeds.
   - **Returns:**
     - `speed`: Linear speed of the robot.
     - `new_angle`: The robot's new estimated angle.

3. **`predict(self, left_motor_speed, right_motor_speed, orientation, x_est_prev, P_est_prev):`**
   - **Purpose:** Predicts the next state estimate using the Kalman filter based on motor speeds, orientation, and previous estimates.
   - **Returns:**
     - `estimated_position`: Estimated position of the robot.
     - `estimated_speed`: Estimated linear speed of the robot.
     - `angle`: Estimated angular speed of the robot.
     - `self.x_est`: Updated state estimate.
     - `self.P_est`: Updated state covariance estimate.

### 6.2 Compensating for Uncertainties<a name="compensating-for-uncertainties"></a>

To enhance the robustness of our localization system, we explicitly consider uncertainties in motor speed measurements and orientation through the process uncertainty matrix (`Q`) and measurement uncertainty matrix (`R`). The values computed in the code are either measured experimentally or extracted from the Practical session number 8 of the course of Mobie Robotics with Professor Francesco Mondada.


## **7 Conclusion**

We methodically created and included key components in this robotics project to imitate the operations of an autonomous fire rescue robot. The robot can determine its position, identify obstacles, and find goals thanks to the Vision Module's interpretation of a specified environment. The A* algorithm drives the Optimal Path Computation, which accurately determines the best path through a grid for the robot's precision navigation. The robot's movements are dynamically adjusted by the Motion Control Implementation, and obstructions detected by proximity sensors are skillfully responded to by the Local Navigation Strategy. In localization, the Kalman Filter improves position estimations by accounting for uncertainties. This study shows how an autonomous fire rescue robot can successfully navigate complex areas, avoid obstacles, and accomplish mission objectives.



